In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Cargar el CSV
df = pd.read_csv("train.csv")

# Usamos solo una muestra para evitar errores de memoria
df_sample = df.head(20000)

# Columnas seleccionadas
selected_columns = [
    "ESTU_PRGM_ACADEMICO",
    "FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEINTERNET",
    "FAMI_EDUCACIONPADRE",
    "FAMI_TIENECOMPUTADOR",
    "coef_1", "coef_2", "coef_3", "coef_4"
]

X_sample = df_sample[selected_columns]
y_sample = df_sample["RENDIMIENTO_GLOBAL"]

# Preprocesamiento
X_sample_filled = X_sample.fillna("desconocido")

cat_cols_simple = [
    "ESTU_PRGM_ACADEMICO",
    "FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEINTERNET",
    "FAMI_EDUCACIONPADRE",
    "FAMI_TIENECOMPUTADOR"
]

num_cols_simple = ["coef_1", "coef_2", "coef_3", "coef_4"]

# Codificamos categóricas
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_cat_encoded = encoder.fit_transform(X_sample_filled[cat_cols_simple])

# Procesamos numéricas
X_num = X_sample_filled[num_cols_simple].replace("na", np.nan).astype(float)
X_num_imputed = SimpleImputer(strategy="mean").fit_transform(X_num)
X_num_scaled = StandardScaler().fit_transform(X_num_imputed)

# Final: datos listos
X_final_sample = np.hstack([X_cat_encoded, X_num_scaled])



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Usa las variables preprocesadas
X_train, X_test, y_train, y_test = train_test_split(
    X_final_sample, y_sample, test_size=0.2, random_state=42, stratify=y_sample
)

# Entrenar modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predicción
y_pred = clf.predict(X_test)

# Cálculo de métricas
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [ ]:
print("Accuracy:", accuracy)
print(report)

Accuracy: 0.3355
              precision    recall  f1-score   support

        alto       0.41      0.46      0.43      1010
        bajo       0.36      0.38      0.37       985
  medio-alto       0.27      0.24      0.25       991
  medio-bajo       0.29      0.26      0.27      1014

    accuracy                           0.34      4000
   macro avg       0.33      0.34      0.33      4000
weighted avg       0.33      0.34      0.33      4000

